# Préparation d'une table de traduction

Étant donné le nombre d'attributs catégoriels utilisés et leurs utilisation sous la forme de valeurs numériques, il est nécessaire d'avoir un fichier permettant de retrouver les catégories associées à ces valeurs. Ce notebook sert donc à la création d'une table de traduction.

Les différentes informations utilisées pour sa création ont été trouvées dans les données elles-mêmes (dans le cas du milieu naturel) inais que dans les fichiers de métadonnées accompagnant les données.

In [1]:
# Imports nécessaires pour ce notebook
import pandas as pd
import geopandas as gpd

## Création du dataframe

Afin de pouvoir mettre l'ensemble des catégories dans une unique table, il est nécessaire d'y avoir le nom de l'attribut associé, afin de pouvoir facilement les utiliser.

In [2]:
df = pd.DataFrame(columns=['Feature', 'Value', 'Category'])

## Ajout des catégories pour les milieux naturels

Le geodataframe pour les milieux naturels contient plusieurs couches, dont l'une est utilisée comme catalogue pour les codes numériques. Il s'agit de la couche `TypoCH_Catalogue`. 

In [3]:
gdb_path = "data/HabitatMap_CH_V1_20220805.gdb"
selected_layer = "TypoCH_Catalogue"

gdf = gpd.read_file(gdb_path, layer=selected_layer)
gdf.head()

,CODETypoCH,DETypoCH,FRTypoCH,ITTypoCH
0,1,1 Gewässer,1 Eaux libres,1 Ambienti acquatici
1,1.1,1.1 Stehende Gewässer,1.1 Eaux calmes,1.1 Acque ferme
2,1.1.0,1.1.0 Stehendes Gewässer ohne Vegetation,1.1.0 Eau sans végétation,1.1.0 Acque prive di vegetazione
3,1.1.0.1,1.1.0.1 Tiefgründiges Gewässer (Freiwasserzone),1.1.0.1 Eau profonde (zone limnétique),1.1.0.1 Acque profonde
4,1.1.0.2,"1.1.0.2 Seichtes Gewässer (Litoral, inkl. Tümpel)",1.1.0.2 Eau peu profonde (incl. mares temporai...,1.1.0.2 Acque poco profonde (comprese le pozze...


N'ayant pris que les classes de niveaux 2 lors de la sélection des données, seules celles-ci vont être sélectionnées, avec la catégorie associée, en Français.

In [4]:
# Identifier les codes de niveau 2 (avec un unique point et sans lettres)
df_env = gdf[gdf['CODETypoCH'].str.contains(r'^\d+\.\d+$')].copy()

# Retirer le point de ce champ
df_env['CODETypoCH'] = df_env['CODETypoCH'].str.replace('.', '', regex=False)

# Suppression des colonnes inutilisées
df_env.drop(columns=['DETypoCH', 'ITTypoCH'], inplace=True)

# Renommage des colonnes et ajout du nom de la feature
df_env.columns = ['Value', 'Category']
df_env['Feature'] = 'TypoCH_NUM'

# Ajouter les nouvelles valeurs extraites au DataFrame final
df = pd.concat([df, df_env], ignore_index=True)

## Ajout des catégories pour les caractéristiques du sol

Malheureusement, les différentes caractéristiques du sol ne sont pas présentes dans la geodatabase. Les classes doivent donc être reprises manuellement depuis les fichiers de métadonnées.

In [5]:
# Ajout des valeurs prises par l'attribut `WASSERSPEI` (= Capacité de rétention hydrique )
values_wasserspei = {
    'Feature': ['WASSERSPEI'] * 8,
    'Value': [9999, 0, 1, 2, 3, 4, 5, 6],
    'Category': ['inconnu', 'inconnu', 'extrêmement faible', 'très faible', 'faible', 'moyen', 'bon', 'très bon']
}

df = pd.concat([df, pd.DataFrame(values_wasserspei)], ignore_index=True)

In [6]:
# Ajout des valeurs prises par l'attribut `NAHRSTOFF` (= Capacité de rétention en substances nutritives )
values_nahrstoff = {
    'Feature': ['NAHRSTOFF'] * 8,
    'Value': [-9999, 0, 1, 2, 3, 4, 5, 6],
    'Category': ['inconnu', 'inconnu', 'extrêmement faible', 'très faible', 'faible', 'moyen', 'bon', 'très bon']
}

df = pd.concat([df, pd.DataFrame(values_nahrstoff)], ignore_index=True)

In [7]:
# Ajout des valeurs prises par l'attribut `WASSERDURC` (= Perméabilité)
values_wasserdurc = {
    'Feature': ['WASSERDURC'] * 7,
    'Value': [-9999, 0, 2, 3, 4, 5, 6],
    'Category': ['inconnu', 'inconnu', 'très ralenti', 'ralenti', 'légèrement ralenti', 'normal', 'excessif']
}

df = pd.concat([df, pd.DataFrame(values_wasserdurc)], ignore_index=True)

## Enregistrement du dataframe

In [8]:
# Sauvegarde du DataFrame dans un fichier CSV
filename = 'data/translation_table.csv'
df.to_csv(filename, index=True, encoding='utf-8')